In [1]:
%cd ../..

/home/dmoreno/ATAT_ZTF/elasticc_one


In [16]:
from src.models.LitATAT import LitATAT
from src.layers import ATAT

import logging
import glob
import torch
import yaml
import copy


from collections import OrderedDict

In [11]:
def handler_checkpoint(path, args):
    od_ = OrderedDict()
    logging.info("Loading model from checkpoint ...")
    checkpoint_ = torch.load(path)
    for key in checkpoint_["state_dict"].keys():
        od_[key.replace("atat.", "")] = checkpoint_["state_dict"][key]
    logging.info("New keys formated ...")
    model = ATAT(**args)
    logging.info("Build ATAT  model")
    try:
        model.load_state_dict(od_, strict=False)
        logging.info("All keys matched")
    except RuntimeError as e:
        logging.error(f"Error loading model state dict: {e}")
    
    return model

def handler_ckpt_path(path):
    out_path = glob.glob(path + "/*.ckpt")[0]
    return out_path

def load_yaml(path):
    with open(path, 'r') as file:
        args = yaml.load(file, Loader=yaml.FullLoader)
    return args

In [23]:
path_model_1 = 'results/ELASTICC_1/LC/lc_tm'

args_1 = load_yaml('{}/args.yaml'.format(path_model_1))

pl_model_1 = LitATAT(**args_1)

pl_model_1.atat = handler_checkpoint(
            handler_ckpt_path(path_model_1), args=args_1
        )

pl_model_1

LitATAT(
  (atat): ATAT(
    (time_encoder): TimeHandler(
      (time_encoders): ModuleList(
        (0): TimeFilmModified(
          (linear_proj): Sequential(
            (0): Linear(in_features=1, out_features=192, bias=True)
          )
        )
        (1): TimeFilmModified(
          (linear_proj): Sequential(
            (0): Linear(in_features=1, out_features=192, bias=True)
          )
        )
        (2): TimeFilmModified(
          (linear_proj): Sequential(
            (0): Linear(in_features=1, out_features=192, bias=True)
          )
        )
        (3): TimeFilmModified(
          (linear_proj): Sequential(
            (0): Linear(in_features=1, out_features=192, bias=True)
          )
        )
        (4): TimeFilmModified(
          (linear_proj): Sequential(
            (0): Linear(in_features=1, out_features=192, bias=True)
          )
        )
        (5): TimeFilmModified(
          (linear_proj): Sequential(
            (0): Linear(in_features=1, out_featur

In [19]:
args_1['general']['num_classes']

20

In [20]:
args_2 = copy.copy(args_1)
args_2['general']['num_classes'] = 22
args_2

{'lc': {'embedding_size': 192,
  'embedding_size_sub': 384,
  'num_heads': 4,
  'num_encoders': 3,
  'Tmax': 1500,
  'num_harmonics': 64,
  'encoder_type': 'Linear',
  'max_pool_kernel': 5,
  'cnn_kernel': 5,
  'pe_type': 'tm',
  'num_bands': 6},
 'ft': {'embedding_size': 128,
  'embedding_size_sub': 256,
  'num_heads': 4,
  'num_encoders': 3,
  'encoder_type': 'Linear',
  'length_size': 0,
  'list_time_to_eval': None},
 'general': {'experiment_type': 'lc',
  'experiment_name': 'lc_tm',
  'name_dataset': 'elasticc_1',
  'data_root': 'data/final/ELASTICC_1/LC_MD_FEAT',
  'use_lightcurves': True,
  'use_metadata': False,
  'use_features': False,
  'batch_size': 256,
  'num_epochs': 10000,
  'patience': 3,
  'lr': 0.001,
  'use_mask_detection': False,
  'use_time_nondetection': False,
  'force_online_opt': False,
  'online_opt_tt': False,
  'use_QT': False,
  'load_pretrained_model': False,
  'src_checkpoint': '.',
  'use_augmented_dataset': False,
  'cambiar_clf': False,
  'num_classes':

In [26]:
pl_model_2 = LitATAT(**args_2)

pl_model_2.atat = handler_checkpoint(
            handler_ckpt_path(path_model_1), args=args_2
        )
pl_model_2.atat.change_clf(args_2['general']["num_classes"])
pl_model_2

ERROR:root:Error loading model state dict: Error(s) in loading state_dict for ATAT:
	size mismatch for classifier_lc.output_layer.weight: copying a param with shape torch.Size([20, 192]) from checkpoint, the shape in current model is torch.Size([22, 192]).
	size mismatch for classifier_lc.output_layer.bias: copying a param with shape torch.Size([20]) from checkpoint, the shape in current model is torch.Size([22]).


LitATAT(
  (atat): ATAT(
    (time_encoder): TimeHandler(
      (time_encoders): ModuleList(
        (0): TimeFilmModified(
          (linear_proj): Sequential(
            (0): Linear(in_features=1, out_features=192, bias=True)
          )
        )
        (1): TimeFilmModified(
          (linear_proj): Sequential(
            (0): Linear(in_features=1, out_features=192, bias=True)
          )
        )
        (2): TimeFilmModified(
          (linear_proj): Sequential(
            (0): Linear(in_features=1, out_features=192, bias=True)
          )
        )
        (3): TimeFilmModified(
          (linear_proj): Sequential(
            (0): Linear(in_features=1, out_features=192, bias=True)
          )
        )
        (4): TimeFilmModified(
          (linear_proj): Sequential(
            (0): Linear(in_features=1, out_features=192, bias=True)
          )
        )
        (5): TimeFilmModified(
          (linear_proj): Sequential(
            (0): Linear(in_features=1, out_featur

In [27]:
# Asumimos que pl_model_1 y pl_model_2 son tus dos modelos a comparar
state_dict_1 = pl_model_1.state_dict()
state_dict_2 = pl_model_2.state_dict()

# Verificar si ambos modelos tienen la misma arquitectura
if state_dict_1.keys() != state_dict_2.keys():
    print("Los modelos tienen arquitecturas diferentes y no pueden compararse directamente.")
else:
    # Iterar sobre cada clave en el state_dict para comparar los pesos
    for key in state_dict_1:
        if torch.equal(state_dict_1[key], state_dict_2[key]):
            print(f"Los pesos de {key} son iguales.")
        else:
            print(f"Los pesos de {key} difieren.")

            # Si deseas saber la magnitud de la diferencia, puedes calcular una métrica como la norma de la diferencia
            difference = torch.norm(state_dict_1[key] - state_dict_2[key]).item()
            print(f"La norma de la diferencia para {key} es: {difference}")

Los pesos de atat.time_encoder.time_encoders.0.alpha_sin son iguales.
Los pesos de atat.time_encoder.time_encoders.0.alpha_cos son iguales.
Los pesos de atat.time_encoder.time_encoders.0.beta_sin son iguales.
Los pesos de atat.time_encoder.time_encoders.0.beta_cos son iguales.
Los pesos de atat.time_encoder.time_encoders.0.ar son iguales.
Los pesos de atat.time_encoder.time_encoders.0.linear_proj.0.weight son iguales.
Los pesos de atat.time_encoder.time_encoders.0.linear_proj.0.bias son iguales.
Los pesos de atat.time_encoder.time_encoders.1.alpha_sin son iguales.
Los pesos de atat.time_encoder.time_encoders.1.alpha_cos son iguales.
Los pesos de atat.time_encoder.time_encoders.1.beta_sin son iguales.
Los pesos de atat.time_encoder.time_encoders.1.beta_cos son iguales.
Los pesos de atat.time_encoder.time_encoders.1.ar son iguales.
Los pesos de atat.time_encoder.time_encoders.1.linear_proj.0.weight son iguales.
Los pesos de atat.time_encoder.time_encoders.1.linear_proj.0.bias son iguales

RuntimeError: The size of tensor a (20) must match the size of tensor b (22) at non-singleton dimension 0